In [14]:
import numpy as np
import cv2 as cv
import math

In [15]:
cap = cv.VideoCapture(0)

#if not cap.isOpened():
    #print("Cannot open camera")
    #exit()

while True:
    
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    frame = cv.flip(frame, 1)
    region_of_interest = frame[100:300, 100:300]
    kernel = np.ones((3,3),np.uint8)
    font = cv.FONT_HERSHEY_DUPLEX

    cv.rectangle(frame, (100, 100), (300, 300), (0, 255, 0), 0)
    hsv = cv.cvtColor(region_of_interest, cv.COLOR_BGR2HSV)
    
    lower_skin = np.array([0,20,70], dtype=np.uint8)      
    upper_skin = np.array([20,255,255], dtype=np.uint8)
    
    mask = cv.inRange(hsv, lower_skin, upper_skin)
    mask = cv.dilate(mask, kernel, iterations = 4)

    _, contours, hierarchy = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    
    
    contour = max(contours, key = lambda x: cv.contourArea(x))
    
    epsilon = 0.0005 * cv.arcLength(contour, True)
    approx = cv.approxPolyDP(contour, epsilon, True)
    hull = cv.convexHull(contour)
    areahull = cv.contourArea(hull)
    areacnt = cv.contourArea(contour)
    arearatio = ((areahull - areacnt) / areacnt) * 100
    hull = cv.convexHull(approx, returnPoints = False)
    defects = cv.convexityDefects(approx, hull)
    
    l = 0
    for i in range(defects.shape[0]):
        s, e, f, d = defects[i, 0]
        start = tuple(approx[s][0])
        end = tuple(approx[e][0])
        far = tuple(approx[f][0])
        
        a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
        b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
        c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
        s = (a + b + c) / 2
        ar = math.sqrt(s * (s - a) * (s - b) * (s - c))
        d = (2 * ar) / a
        angle = math.acos((b ** 2 + c ** 2 - a ** 2)/(2 * b * c)) * 57
        
        if angle <= 90:
            l += 1
    l += 1
                
    if l == 1:
        if areacnt < 2000:
            cv.putText(frame, 'Put your hand in the box.', (0, 50), font, 0, (0, 0, 255), 3, cv.LINE_AA)
        else:
            if arearatio == 10:
                cv.putText(frame, 'Zero.', (0, 50), font, 2, (0, 0, 255), 3, cv.LINE_AA)
            else:
                cv.putText(frame, 'One.', (0, 50), font, 2, (0, 0, 255), 3, cv.LINE_AA)
            
    if l == 2:
        cv.putText(frame, 'Two.', (0, 50), font, 0, (0, 0, 255), 3, cv.LINE_AA)
        
    if l == 3:
        cv.putText(frame, 'Three.', (0, 50), font, 2, (0, 0, 255), 3, cv.LINE_AA)
            
    if l == 4:
        cv.putText(frame, 'Four.', (0, 50), font, 2, (0, 0, 255), 3, cv.LINE_AA)
            
    if l == 5:
        cv.putText(frame, 'Five.', (0, 50), font, 2, (0, 0, 255), 3, cv.LINE_AA)
        
    else:
        cv.putText(frame, 'Cannot detect hand.', (0, 50), font, 0, (0, 0, 255), 3, cv.LINE_AA)
        
    cv.imshow('mask', mask)
    cv.imshow('frame', frame)
    if cv.waitKey(1) == ord('q'):
        break
# When everything done, release the capture
cap.release()
cv.destroyAllWindows()